In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tqdm.auto import tqdm
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Train.csv")

train_df = pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Train.csv")
test_df = pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Test.csv")

In [ ]:
print(train_df.shape)

In [ ]:
train_df.head()

In [ ]:
train_df.columns

In [ ]:
df = train_df

In [ ]:
df.duplicated().sum()

In [ ]:
# df.isnull().sum(axis = 0)

In [ ]:
df['class'].value_counts()

In [ ]:
# df.describe().T

In [ ]:
df_cols = []
for column in df.columns:
    if df[column].nunique() > 100:
        df_cols.append(column)

In [ ]:
sns.set_style("darkgrid") 
plt.figure(figsize = (8,25))
for idx, col in enumerate(df_cols, 1):
    # plt.subplot(len(df_cols) , 3 , 3*idx -2 )
    # plt.title(f"{col} | Skewness: {round(df[col].skew(), 2)}") 
    # sns.histplot(df[col], kde = True)
    plt.subplot(len(df_cols), 2 , 2*idx - 1 )
    sns.boxplot(x = df[col])
    plt.subplot(len(df_cols), 2 , 2*idx  )
    sns.scatterplot(x=col, y='class', data=df, hue='class')
plt.tight_layout()
plt.show() 

In [ ]:
def replace_outliers_with_median(df):
    for column in df.columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        median_value = df[column].median()
        percentile_90 = df[column].quantile(0.9)
        percentile_10 = df[column].quantile(0.1)
        
        # Define the outlier thresholds
        upper_bound = Q3 + 1.5 * IQR + percentile_90
        lower_bound = Q1 - 1.5 * IQR - percentile_10

        # Replace outliers with the median
        df[column] = df[column].apply(lambda x: median_value if x > upper_bound or x < lower_bound else x)

    return df

In [ ]:
df = replace_outliers_with_median(df)

In [ ]:
Q3 = df.quantile(0.75)
Q1 = df.quantile(0.25)
IQR = Q3 - Q1

In [ ]:
mask = (df > (Q3 + 1.5*IQR)) | (df < (Q1 - 1.5*IQR))

In [ ]:
# mask.sum(axis = 0)

In [ ]:
# max(mask.sum(axis = 1))

In [ ]:
sum(mask.sum(axis = 1) > 3)

In [ ]:
df = df.loc[mask.sum(axis = 1) <= 3 , :]

In [ ]:
df['class'].value_counts()

In [ ]:
Q3 = df.quantile(0.75)
Q1 = df.quantile(0.25)
IQR = Q3 - Q1

In [ ]:
mask = (df > (Q3 + 1.5*IQR)) | (df < (Q1 - 1.5*IQR))


In [ ]:
# mask.sum(axis = 0)

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
sns.set_style("darkgrid") 
plt.figure(figsize = (8,25))
for idx, col in enumerate(df_cols, 1):
    # plt.subplot(len(df_cols) , 3 , 3*idx -2 )
    plt.subplot(len(df_cols), 2 , 2*idx - 1 )
    sns.boxplot(x = df[col])
    plt.subplot(len(df_cols), 2 , 2*idx  )
    plt.title(f"{col} | Skewness: {round(df[col].skew(), 2)}") 
    sns.histplot(df[col], kde = True)
plt.tight_layout()
plt.show() 

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(df.corr(), annot = True)

In [ ]:
pca_cols = [col for col in df.columns if abs(df.corr()['class'][col]) < 0.01]

In [ ]:
pca_cols

In [ ]:
df.columns

In [ ]:
df.drop(columns = pca_cols, inplace = True, axis = 1)

In [ ]:
df.drop(columns = ['V4'], inplace = True, axis = 1)

In [ ]:
mask.sum(axis = 0).loc[pca_cols]

In [ ]:
# X.shape

# Applying Models

In [ ]:
X = df.drop(['class'], axis=1)
y = df['class']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25)

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
selector = SelectKBest(score_func=f_classif, k=10)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_val)  

In [ ]:
scaler = RobustScaler()
X_train_robust = scaler.fit_transform(X_train_selected)
X_test_robust = scaler.transform(X_test_selected)

In [ ]:
quantile_scaler = QuantileTransformer()
X_train_quantile = quantile_scaler.fit_transform(X_train)
X_test_quantile = quantile_scaler.transform(X_val)

## Random Forest Classifier 
###### It is performing well till now

In [ ]:
rfc = RandomForestClassifier(max_depth = 8,criterion='entropy')

In [ ]:
rfc.fit(X_train_robust, y_train)

In [ ]:
print(classification_report(y_train, rfc.predict_proba(X_train_robust)[:, 1] >= 0.47))

In [ ]:
prediction = rfc.predict_proba(X_test_robust)

In [ ]:
f1_score(y_val, prediction[:, 1] >= 0.47)

In [ ]:
print(classification_report(y_val, prediction[:,1] >= 0.5))

## XGB Classifier

In [ ]:
xgb = XGBClassifier(max_depth = 20)

In [ ]:
xgb.fit(X_train_robust, y_train)

In [ ]:
prediction_xgb = (xgb.predict_proba(X_test_robust)[:,1] >= 0.5).astype(int)

In [ ]:
f1_score(y_val, prediction_xgb)

In [ ]:
print(classification_report(y_val, prediction_xgb))

## SVC Algorithm (I think it will perform well ,#gut felling)

In [ ]:
svc = svm.SVC()

In [ ]:
svc.fit(X_train_quantile, y_train)

In [ ]:
prediction_svc = svc.predict(X_test_quantile)

In [ ]:
f1_score(y_val, prediction_svc)

In [ ]:
parameters = {'C': [ 0.005, 0.01, 0.05, 0.5, 5, 10, 50, 1000],
              'gamma':[0.001, 0.01, 0.1, 0.5, 1],
              'kernel': ['rbf']
              }    

In [ ]:
cl = svm.SVC()
grid = GridSearchCV(cl, parameters, cv=10)
grid.fit(X_train_quantile, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_estimator_)

In [ ]:
grid_prediction = grid.predict(X_test_quantile)

In [ ]:
f1_score(y_val, grid_prediction)

In [ ]:
parameters2 = {'C': [5],
              'gamma':[0.05, 0.5],
              'kernel': ['rbf']
              }  

In [ ]:
cl2 = svm.SVC()
grid2 = GridSearchCV(cl2, parameters2, cv=10)
grid2.fit(X_train_quantile, y_train)

In [ ]:
print(grid2.best_params_)
print(grid2.best_estimator_)

In [ ]:
grid_prediction = grid2.predict(X_test_quantile)

In [ ]:
f1_score(y_val, grid_prediction)

# Applying on the test data

In [ ]:
test_data = pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Test.csv")

In [ ]:
Index=test_data['Index']
test_data.drop(['Index'],axis=1,inplace=True)

In [ ]:
test_robust = scaler.transform(test_data)

In [ ]:

y_test = rfc.predict_proba(test_robust)

In [ ]:
y_test= y_test[:, 1] > 0.47

In [ ]:
result = pd.DataFrame({
    'Index': Index,
    'class': y_test.astype('int')
})

In [ ]:
result

In [ ]:
result.to_csv("submission3.csv", index=False) #(for making submission file)